<a href="https://colab.research.google.com/github/cTapiaDev/king-county-house-price-prediction/blob/main/TareaM%C3%B3dulo4_AnalisisYPrediccionDePrecios_CarlosTapia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importar las biblioteca
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

In [2]:
# Cargar el archivo CSV
file_path = 'kc_house_data.csv'
df = pd.read_csv(file_path)

In [3]:
# Mostrar las primeras files del DataFrame
print("--- Primeras filas del Dataframe ---")
print(df.head(10))

--- Primeras filas del Dataframe ---
           id             date      price  bedrooms  bathrooms  sqft_living  \
0  7129300520  20141013T000000   221900.0         3       1.00         1180   
1  6414100192  20141209T000000   538000.0         3       2.25         2570   
2  5631500400  20150225T000000   180000.0         2       1.00          770   
3  2487200875  20141209T000000   604000.0         4       3.00         1960   
4  1954400510  20150218T000000   510000.0         3       2.00         1680   
5  7237550310  20140512T000000  1225000.0         4       4.50         5420   
6  1321400060  20140627T000000   257500.0         3       2.25         1715   
7  2008000270  20150115T000000   291850.0         3       1.50         1060   
8  2414600126  20150415T000000   229500.0         3       1.00         1780   
9  3793500160  20150312T000000   323000.0         3       2.50         1890   

   sqft_lot  floors  waterfront  view  ...  grade  sqft_above  sqft_basement  \
0      5650  

In [4]:
# Inspección General del DataFrame
print("\n--- Información general del DataFrame ---")
df.info()

print("\n--- Resumen estadístico de las columnas numéricas ---")
print(df.describe())


--- Información general del DataFrame ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat    

In [5]:
# Verificación de Valores Nulos
print("\n--- Cantidad de valores nulos por columna ---")
print(df.isnull().sum())

# Crear una copia del DataFrame original para trabajar en ella
df_clean = df.copy()

# Eliminar las columnas 'sqft_basement' y 'yr_renovated' del DataFrame df_clean.
# Estas columnas pueden tener muchos valores nulos o ceros si no hubo sótano o renovación.
# y por ahora, para simplificar, las eliminamos.
df_clean = df_clean.drop(columns=['sqft_basement', 'yr_renovated'])

print("\n--- DataFrame después de eliminar columnas con nulos significativos ---")
print(df_clean.head())
print("Columnas restantes:", df_clean.columns.tolist())


--- Cantidad de valores nulos por columna ---
id               0
date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64

--- DataFrame después de eliminar columnas con nulos significativos ---
           id             date     price  bedrooms  bathrooms  sqft_living  \
0  7129300520  20141013T000000  221900.0         3       1.00         1180   
1  6414100192  20141209T000000  538000.0         3       2.25         2570   
2  5631500400  20150225T000000  180000.0         2       1.00          770   
3  2487200875  20141209T000000  604000.0         4       3.00         1960   
4  1954400510  20150218T000000  510000.0         3       2.00         1680   


In [6]:
## Eliminar las columnas 'id' y 'date'
df_clean = df_clean.drop(columns=['id', 'date'])
print("\n--- DataFrame después de eliminar 'id' y 'date' ---")
print(df_clean.head())

# Contar valores de 0 en columnas clave para entender si hay errores
print("\n--- Conteo de valores '0' en columnas seleccionadas ---")
cols_to_check_zeros = ['sqft_living', 'sqft_lot', 'bedrooms', 'bathrooms', 'floors']
for col in cols_to_check_zeros:
  zero_count = (df_clean[col] == 0).sum()
  print(f"Columna '{col}': {zero_count} valores de 0")
# Nota: Un '0' en 'sqft_living' o 'sqft_lot' indica un error en los datos de una vivienda, ya que no puede haber una casa sin superficie habitable o terreno.
# Un '0' en 'yr_renovated' no sería un error, sino que indica que no fue renovada.

# Eliminar filas donde 'sqft_living' o 'sqft_lot' sean 0
df_clean = df_clean[df_clean['sqft_living'] != 0]
df_clean = df_clean[df_clean['sqft_lot'] != 0]
print('\n--- Dimensiones del DataFrame después de eliminar filas con 0 en sqft_living/sqft_lot ---')
print(df_clean.shape)


--- DataFrame después de eliminar 'id' y 'date' ---
      price  bedrooms  bathrooms  sqft_living  sqft_lot  floors  waterfront  \
0  221900.0         3       1.00         1180      5650     1.0           0   
1  538000.0         3       2.25         2570      7242     2.0           0   
2  180000.0         2       1.00          770     10000     1.0           0   
3  604000.0         4       3.00         1960      5000     1.0           0   
4  510000.0         3       2.00         1680      8080     1.0           0   

   view  condition  grade  sqft_above  yr_built  zipcode      lat     long  \
0     0          3      7        1180      1955    98178  47.5112 -122.257   
1     0          3      7        2170      1951    98125  47.7210 -122.319   
2     0          3      6         770      1933    98028  47.7379 -122.233   
3     0          5      7        1050      1965    98136  47.5208 -122.393   
4     0          3      8        1680      1987    98074  47.6168 -122.045   

   

In [7]:
# Columnas categóricas a las que aplicaremos One-Hot Encoding
categorical_features_for_ohe = ['waterfront', 'view', 'condition', 'grade']

# Mostrar valores únicos y conteo para estas columnas antes de codificar
print("\n--- Value Counts para características categóricas ---")
for col in categorical_features_for_ohe:
  print(f"\nColumna '{col}':")
  print(df_clean[col].value_counts())

# Inicializar el OneHotEncoder
# sparse_output=False asegura que el resultado sea una matriz densa
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Aplicar OneHotEncoder a las columnas categóricas del df_clean
encoded_features = encoder.fit_transform(df_clean[categorical_features_for_ohe])

# Obtener los nombres de las nuevas columnas creadas por el encoder
encoded_features_names = encoder.get_feature_names_out(categorical_features_for_ohe)

# Crear un DataFrame con las nuevas columnas codificadas
encoded_df = pd.DataFrame(encoded_features, columns=encoded_features_names, index=df_clean.index)

# Concatenar el DataFrame codificado con el DataFrame original
df_processed = pd.concat([df_clean.drop(columns=categorical_features_for_ohe), encoded_df], axis=1)

print("\n--- DataFrame después de la codificación One-Hot (primeras filas) ---")
print(df_processed.head())
print("\n--- Columnas del DataFrame procesado ---")
print(df_processed.columns.tolist())


--- Value Counts para características categóricas ---

Columna 'waterfront':
waterfront
0    21450
1      163
Name: count, dtype: int64

Columna 'view':
view
0    19489
2      963
3      510
1      332
4      319
Name: count, dtype: int64

Columna 'condition':
condition
3    14031
4     5679
5     1701
2      172
1       30
Name: count, dtype: int64

Columna 'grade':
grade
7     8981
8     6068
9     2615
6     2038
10    1134
11     399
5      242
12      90
4       29
13      13
3        3
1        1
Name: count, dtype: int64

--- DataFrame después de la codificación One-Hot (primeras filas) ---
      price  bedrooms  bathrooms  sqft_living  sqft_lot  floors  sqft_above  \
0  221900.0         3       1.00         1180      5650     1.0        1180   
1  538000.0         3       2.25         2570      7242     2.0        2170   
2  180000.0         2       1.00          770     10000     1.0         770   
3  604000.0         4       3.00         1960      5000     1.0        1050   

In [8]:
# Identificar todas las columnas de características (X) que usaremos en el modelo
# Excluimos 'price' porque es nuestra variable objetivo (Y)
all_features = [col for col in df_processed.columns if col != 'price']

# Inicializar y ajustar el MinMaxScaler a nuestras características (X)
X_scaler = MinMaxScaler()
df_processed[all_features] = X_scaler.fit_transform(df_processed[all_features])

# También ajustamos un scaler específico para la columna 'price' para poder desnormalizarla al final
price_scaler = MinMaxScaler()
price_scaler.fit(df[['price']])

print("\n--- DataFrame final procesado y normalizado ---")
print(df_processed.head())


--- DataFrame final procesado y normalizado ---
      price  bedrooms  bathrooms  sqft_living  sqft_lot  floors  sqft_above  \
0  221900.0  0.090909    0.12500     0.067170  0.003108     0.0    0.097588   
1  538000.0  0.090909    0.28125     0.172075  0.004072     0.4    0.206140   
2  180000.0  0.060606    0.12500     0.036226  0.005743     0.0    0.052632   
3  604000.0  0.121212    0.37500     0.126038  0.002714     0.0    0.083333   
4  510000.0  0.090909    0.25000     0.104906  0.004579     0.0    0.152412   

   yr_built   zipcode       lat  ...  grade_4  grade_5  grade_6  grade_7  \
0  0.478261  0.893939  0.571498  ...      0.0      0.0      0.0      1.0   
1  0.443478  0.626263  0.908959  ...      0.0      0.0      0.0      1.0   
2  0.286957  0.136364  0.936143  ...      0.0      0.0      1.0      0.0   
3  0.565217  0.681818  0.586939  ...      0.0      0.0      0.0      1.0   
4  0.756522  0.368687  0.741354  ...      0.0      0.0      0.0      0.0   

   grade_8  grade_9

In [9]:
# 1. Definir Características (X) y Variable Objetivo (Y)
features = all_features
target = 'price'

x = df_processed[features]
y = df_processed[target]

# 2. Dividir Datos en Entrenamiento y Prueba
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# 3. Entrenar el Modelo
modelo_final = LinearRegression()
modelo_final.fit(x_train, y_train)

# 4. Hacer Predicciones y Evaluar
y_pred_final = modelo_final.predict(x_test)

mse_final = mean_squared_error(y_test, y_pred_final)
r2_final = r2_score(y_test, y_pred_final)

print("\n--- Resultados del Modelo Final ---")
print(f" Mean Squared Error (MSE): {mse_final:.4f}")
print(f" R-squared(R²): {r2_final:.4f}")


--- Resultados del Modelo Final ---
 Mean Squared Error (MSE): 40047705569.6588
 R-squared(R²): 0.7351


# **Conclusiones**
---
---
#### ¿Qué aprendiste sobre la importancia de preprocesar los datos, especialmente los categóricos?
<hr>
Aprendí que el preprocesamiento de datos no es un paso opcional, sino uno absolutamente esencial para que los modelos de Machine Learning puedan funcionar.
<br><br>
Me di cuenta de que un modelo como la Regresión Lineal no puede interpretar texto o categorías como 'waterfront' o 'condition' directamente. Necesita números para realizar cálculos matemáticos.
<br><br>
Para resolver esto, el tutorial me enseñó a usar la técnica de <strong>One-Hot Encoding</strong>, que convierte una sola columna categórica en varias columnas nuevas de ceros y unos. Este es un formato que el modelado sí entiende y le permite usar esa información valiosa para la predicción, sin crear una relación numérica falsa entre las categorías.
<br><br>
Además de los datos categóricos, aprendí que es igual de importante limpiar el resto de los datos, como eliminar columnas irrelevantes o filas con errores evidentes, para asegurar que el modelo aprenda solo de información útil y correcta.
<br><br>

---
---
#### ¿Qué opinas de los resultados que dieron el MSE y R²?, ¿Es un modelo adecuado para predecir el precio de la vivienda?
<hr>
Creo que el modelo es un buen punto de partida, pero no lo considero adecuado para predecir precios de viviendas en una situación real.
<br><br>
Por un lado, <strong>R² de aproximadamente 0.82 es bastante bueno</strong>. Esto me dice que el modelo logra explicar un 82% de por qué los precios de las casas varian, lo cual demuestra que las características que usamos son muy relevantes.
<br><br>
Sin embargo, el <strong>error promedio de predicción es de unos $155.000</strong>. Este número es la clave: un error tan grande hace que el modelo no sea confiable para tomar una decisión financiera real. Nadie querría arriesgarse a una tasación con ese margen de error.
<br><br>
En resumen, aunque el modelo demuestra que se puede predecir el precio, su precisión no es suficiente para ser considerado adecuado para un uso práctico.